Notebook de gestion du monitoring de databricks. 

Dans le cluster, la librairie "opencensus-ext-azure==1.0.7" doit être installée, et la variable d'environnement APPINSIGHTS_CONNEXION_STRING doit exister.

La classe Monitor permet de faire un log classique, un log avec les métriques pour un chargement de données, un log avec les métriques pour la suppression de doublons.

In [0]:
import os, sys
import logging
import time
from opencensus.ext.azure.log_exporter import AzureLogHandler
from enum import Enum
from datetime import datetime

ENVIRONMENT = os.getenv('ENVIRONMENT')

# Type de log
class LIN_LogType(Enum):
  LOG = 1
  LOADING_METRICS = 2
  CLEANING_METRICS = 3

# Niveau de log
class LIN_Level(Enum):
  CRITICAL = 1
  ERROR = 2
  WARNING = 3
  INFO = 4
  DEBUG = 5

# Etape du log LOADING_METRICS
class LIN_Stage(Enum):
  BRONZE = 1
  SILVER = 2
  DATAWAREHOUSE = 3
  DATAMART = 4
  NONE = 5
  
# Source du log LOADING_METRICS
class LIN_Source(Enum):
  GL_LEGACY = 1
  KHEOPS = 2
  MANUEL_REFERENTIEL = 3
  MANUEL_TRANSCO = 4
  MANUEL_CORRECTION = 5
  NONE = 6
  ARTICLE_DB2 = 7
  CLIENT_DB2 = 8
  CLIENT_IDIL = 9
  CLIENT_MANUEL = 10
  CLIENT_PARAM_EDI = 11
  
# Statut du log LOADING_METRICS
class LIN_Status(Enum):
  SUCCESS = 1
  FAILURE = 2

def LIN_get_error_message() -> str:
  # Retourne le type et la valeur de sys.exc_info.
  return str(sys.exc_info()[0]) + " " + str(sys.exc_info()[1])

def LIN_get_source(path: str) -> HAC_Source:
  # Retourne la source en se basant sur un chemin.  
  if '/GLLegacy/' in path:
    return HAC_Source.GL_LEGACY
  elif '/Kheops/' in path:
    return HAC_Source.KHEOPS
  elif '/Manuel/Referentiel/' in path:
    return HAC_Source.MANUEL_REFERENTIEL
  elif '/Manuel/Transco/' in path:
    return HAC_Source.MANUEL_TRANSCO
  elif '/Manuel/Correction/' in path:
    return HAC_Source.MANUEL_CORRECTION
  elif '/Article/db2/' in path:
    return HAC_Source.ARTICLE_DB2
  elif '/Client/db2/' in path:
    return HAC_Source.CLIENT_DB2
  elif '/Client/idil/' in path:
    return HAC_Source.CLIENT_IDIL
  elif '/Client/manuel/' in path:
    return HAC_Source.CLIENT_MANUEL
  elif '/Client/param_edi/' in path:
    return HAC_Source.CLIENT_PARAM_EDI

  return HAC_Source.NONE


In [0]:
class LIN_Monitor():
  
  __logger__: logging.Logger
  __notebook__: str
  __data_factory_name__: str
  __pipeline_run_id__: str
  __pipeline_name__: str
  __source__: str
  __stage__: str
  
  def __init__(self, notebook: str, data_factory_name:str, pipeline_run_id: str, pipeline_name: str, source: HAC_Source, stage: HAC_Stage):
    # Constructeur de la classe.
    # Creation du logger dont le nom est le nom du notebook.
    # notebook (str), nom du notebook.
    # data_factory_name (str), nom de la ressource ADF.
    # pipeline_run_id (str), id de l'exécution du pipeline ADF.
    # pipeline_name (str), nom du pipeline ADF.
    # source (HAC_Source), source de données.
    # stage (HAC_Stage), etape du chargement.
    
    if not isinstance(stage, HAC_Stage):
      raise TypeError('stage parameter must be an instance of HAC_Stage')
      
    if not isinstance(source, HAC_Source):
      raise TypeError('source parameter must be an instance of HAC_Source')
    
    self.__notebook__ = notebook
    self.__source__ = source
    self.__stage__ = stage
    self.__data_factory_name__ = data_factory_name
    self.__pipeline_run_id__ = pipeline_run_id
    self.__pipeline_name__ = pipeline_name
    logger_name = "Monitor_" + notebook + str(datetime.now())

    if not(logging.root.manager.loggerDict.get(logger_name) is None):
         print("Monitor class has been instantiated twice.")
        
    self.__logger__ = logging.getLogger(logger_name)
    azure_log_handler = AzureLogHandler(connection_string=os.getenv('APPINSIGHTS_CONNEXION_STRING'), export_interval=0.0)
    self.__logger__.addHandler(azure_log_handler)
    
  def __del__(self): 
    # Destructeur de la classe.
    # Pour éviter le problème en recette en executant le notebook avec ADF, ou aucun log n'est produit, même avec export_interval=0.0.
    time.sleep(5)
    
  def log(self, level: HAC_Level, message: str):
    # Creation d'un log basique avec le choix du niveau de log.
    # level (HAC_Level), niveau de log.
    # message (str), message du log.
      
    if not isinstance(level, HAC_Level):
      raise TypeError('level parameter must be an instance of HAC_Level')
    
    properties = {'custom_dimensions': {
      'dataFactoryName': self.__data_factory_name__,
      'environment': ENVIRONMENT, 
      'logType': HAC_LogType.LOG.name, 
      'notebookName': self.__notebook__,
      'pipelineName': self.__pipeline_name__,
      'pipelineRunId': self.__pipeline_run_id__}}
    
    cleaned_message = message.replace("\n", " ")
    
    self.__logger__.setLevel(logging.DEBUG)
    if level == HAC_Level.CRITICAL:
      self.__logger__.critical(cleaned_message, extra=properties)
    elif level == HAC_Level.ERROR:
      self.__logger__.error(cleaned_message, extra=properties)
    elif level == HAC_Level.WARNING:
      self.__logger__.warning(cleaned_message, extra=properties)
    elif level == HAC_Level.INFO:
      self.__logger__.info(cleaned_message, extra=properties)
    elif level == HAC_Level.DEBUG:
      self.__logger__.debug(cleaned_message, extra=properties)
    self.__logger__.setLevel(logging.WARNING)
    
  def log_loading_metrics(self, status: HAC_Status, input_path: str, output_path: str, start_timestamp: datetime, end_timestamp: datetime, message: str, output_rows_count: int=0, input_file_size: int=0, output_file_size: int=0):
    # Creation d'un log avec les informations d'un chargement de fichier.
    # status (str), statut du chargement.
    # input_path (str), chemin source.
    # output_path (str), chemin destination.
    # start_timestamp (datetime), date et heure de debut du chargement.
    # end_timestamp (datetime), date et heure de fin du chargement.
    # message (int), message d'erreur. 
    # output_rows_count (int), nombre de lignes chargees.
    # input_file_size (int), taille du fichier source.
    # output_file_size (int), taille du fichier destination.
      
    if not isinstance(status, HAC_Status):
      raise TypeError('status parameter must be an instance of HAC_Status')
    
    cleaned_message = message.replace("\n", " ")
    
    properties = {'custom_dimensions': {
      'dataFactoryName': self.__data_factory_name__,
      'duration': (end_timestamp - start_timestamp).total_seconds(), 
      'endLoading': end_timestamp.strftime("%Y-%m-%d %H:%M:%S.%f"), 
      'environment': ENVIRONMENT, 
      'inputFileSize': input_file_size,
      'inputPath': input_path, 
      'logType': HAC_LogType.LOADING_METRICS.name, 
      'notebookName': self.__notebook__,
      'message': cleaned_message,
      'outputFileSize': output_file_size,
      'outputPath': output_path,
      'pipelineName': self.__pipeline_name__,
      'pipelineRunId': self.__pipeline_run_id__,
      'rowCount': output_rows_count,
      'source': self.__source__.name,
      'stage': self.__stage__.name,
      'startLoading': start_timestamp.strftime("%Y-%m-%d %H:%M:%S.%f"),
      'status': status.name}}
    
    self.__logger__.setLevel(logging.DEBUG)
    if status == HAC_Status.SUCCESS:
      self.__logger__.info(HAC_LogType.LOADING_METRICS.name, extra=properties)
    else:
      self.__logger__.error(HAC_LogType.LOADING_METRICS.name, extra=properties)
    self.__logger__.setLevel(logging.WARNING)
    
  def log_cleaning_metrics(self, status: HAC_Status, table_path: str, start_timestamp: datetime, end_timestamp: datetime, message: str, deleted_rows_count: int=0):
    # Creation d'un log avec les informations de suppression d'une table delta.
    # table_path (str), table nettoyee.
    # start_timestamp (datetime), date et heure de debut du chargement.
    # end_timestamp (datetime), date et heure de fin du chargement.
    # deleted_rows_count (int), nombre de lignes supprimees 
    
    cleaned_message = message.replace("\n", " ")
    
    properties = {'custom_dimensions': {
      'dataFactoryName': self.__data_factory_name__,
      'duration': (end_timestamp - start_timestamp).total_seconds(), 
      'endLoading': end_timestamp.strftime("%Y-%m-%d %H:%M:%S.%f"), 
      'environment': ENVIRONMENT, 
      'logType': HAC_LogType.CLEANING_METRICS.name, 
      'message': cleaned_message,
      'notebookName': self.__notebook__,
      'pipelineName': self.__pipeline_name__,
      'pipelineRunId': self.__pipeline_run_id__,
      'rowCount': deleted_rows_count,
      'startLoading': start_timestamp.strftime("%Y-%m-%d %H:%M:%S.%f"),
      'tablePath': table_path,
      'status': status.name}}
    
    self.__logger__.setLevel(logging.DEBUG)
    self.__logger__.info(HAC_LogType.CLEANING_METRICS.name, extra=properties)
    self.__logger__.setLevel(logging.WARNING)